<a href="https://colab.research.google.com/github/pri-nitta/FIAP_IA/blob/main/gerando_um_modelo_de_classifica%C3%A7%C3%A3o_com_IA_generativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.datasets import load_iris
iris_data = load_iris()

In [3]:
iris_df = pd.DataFrame(data = iris_data['data'], columns = iris_data['feature_names'])
iris_df['Target'] = iris_data['target']
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
set(iris_df['Target'])

{0, 1, 2}



1.   Verificação de dados nulos
2.   Separar os dados em treino e teste
4. normalizar os dados
4. treinaro o modelo (ramdon forest)
5. validar o modelo (classification report)





In [5]:
print(iris_df.isnull().sum())

# Tratamento de dados nulos (se existirem)
# Exemplo: Substituindo valores nulos pela média da coluna
# for col in iris_df.columns:
#  if iris_df[col].isnull().any():
#    iris_df[col].fillna(iris_df[col].mean(), inplace=True)

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
Target               0
dtype: int64


In [6]:
# prompt: como posso separar os dados em treino e teste?

from sklearn.model_selection import train_test_split

# Separando os dados em treino e teste
X = iris_df.drop('Target', axis=1) # dropar a coluna target
y = iris_df['Target'] #variavel dependente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 70% treino e 30% teste

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (105, 4)
Shape of X_test: (45, 4)
Shape of y_train: (105,)
Shape of y_test: (45,)


In [7]:
# prompt: normalize os dados de treino e teste

from sklearn.preprocessing import MinMaxScaler #normalizando em 0 e 1

# Normalizando os dados de treino e teste
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Shape of X_test_scaled:", X_test_scaled.shape)

Shape of X_train_scaled: (105, 4)
Shape of X_test_scaled: (45, 4)


In [8]:
print(X_train_scaled[:5])

[[0.35294118 0.18181818 0.46428571 0.375     ]
 [0.58823529 0.36363636 0.71428571 0.58333333]
 [0.61764706 0.5        0.78571429 0.70833333]
 [0.67647059 0.45454545 0.58928571 0.54166667]
 [0.85294118 0.72727273 0.89285714 1.        ]]


In [9]:
# prompt: treine o modelo com o random forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Treinando o modelo Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42) #n_estimators é o número de árvores na floresta
rf_classifier.fit(X_train_scaled, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_classifier.predict(X_test_scaled)

# Avaliando o modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



resultado foi overfitting.

In [10]:
# prompt: o modelo gerou overfitting, melhore o modelo

# Importando bibliotecas necessárias (se ainda não estiverem importadas)
from sklearn.model_selection import GridSearchCV

# Definindo os parâmetros para o GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Variando o número de árvores
    'max_depth': [None, 10, 20],     # Variando a profundidade máxima das árvores
    'min_samples_split': [2, 5, 10], # Variando o número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4]    # Variando o número mínimo de amostras em cada folha
}

# Criando o modelo RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Criando o GridSearchCV
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Executando o GridSearchCV nos dados de treinamento
grid_search.fit(X_train_scaled, y_train)

# Imprimindo os melhores parâmetros encontrados
print("Melhores parâmetros:", grid_search.best_params_)

# Treinando o modelo com os melhores parâmetros
best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_train_scaled, y_train)

# Fazendo previsões com o melhor modelo
y_pred = best_rf_classifier.predict(X_test_scaled)

# Avaliando o modelo com os melhores parâmetros
print(classification_report(y_test, y_pred))

Melhores parâmetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [11]:
# prompt: avalie o resultado com o classification report usando o grid search best estimator

# A avaliação do modelo com o classification_report já está presente no código fornecido.
# A linha abaixo imprime o classification_report usando o melhor estimador do GridSearchCV.
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [12]:
# prompt: o modelo ainda está com overfitting, teste um modelo menos complexo

from sklearn.linear_model import LogisticRegression

# Crie e treine um modelo de Regressão Logística
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train_scaled, y_train)

# Faça previsões no conjunto de teste
y_pred_logistic = logistic_model.predict(X_test_scaled)

# Avalie o modelo
print(classification_report(y_test, y_pred_logistic))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.69      0.82        13
           2       0.76      1.00      0.87        13

    accuracy                           0.91        45
   macro avg       0.92      0.90      0.89        45
weighted avg       0.93      0.91      0.91        45



In [13]:
# prompt: teste com decision tree

from sklearn.tree import DecisionTreeClassifier

# Crie e treine um modelo de Árvore de Decisão
decision_tree_model = DecisionTreeClassifier(max_depth=2, random_state=42)
decision_tree_model.fit(X_train_scaled, y_train)

# Faça previsões no conjunto de teste
y_pred_dt = decision_tree_model.predict(X_test_scaled)

# Avalie o modelo
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [14]:
#incluindo max depth
decision_tree_model = DecisionTreeClassifier(max_depth=2, random_state=42)
decision_tree_model.fit(X_train_scaled, y_train)

# Faça previsões no conjunto de teste
y_pred_dt = decision_tree_model.predict(X_test_scaled)

# Avalie o modelo
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.92      0.96        13
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45

